# First Jupyter Notebook with the Bluesky Framework : 2021-03 Training Edition

TODO:

See also [Guide: First Steps with Bluesky](https://github.com/BCDA-APS/use_bluesky/blob/main/first_steps_guide.md), a quick-reference guide for those learning how to use the Bluesky Framework for data acquisition.  This guide provides the basic commands you will need when using Bluesky from an IPython console or a Jupyter notebook.

## Start the `instrument` package

In [1]:
from instrument.collection import *

I Tue-12:40:47 - ############################################################ startup
I Tue-12:40:47 - logging started
I Tue-12:40:47 - logging level = 10
I Tue-12:40:47 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/collection.py
I Tue-12:40:47 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Tue-12:40:47 - bluesky framework
I Tue-12:40:47 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/check_python.py
I Tue-12:40:47 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/check_bluesky.py
I Tue-12:40:48 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/initialize.py
/home/mintadmin/Apps/anaconda/envs/bluesky_2021_1/lib/python3.8/site-packages/databroker/core.py:1919: UserWarning: There are 2 entrypoints for the databroker handler spec 'NPY_SEQ'. They are [EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')), EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', None)]. The match EntryPoint('NPY_SEQ', 'ophyd.sim', 'NumpySeqHandler', Distribution('ophyd', '1.6.0')) has won the race.
  warnings.warn(
I Tue-12:40:49 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/metad

## Where _is_ everybody?

On the command line, there is an [IPython magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#built-in-magic-commands) command from the [bluesky](https://blueskyproject.io/bluesky/) package to print the current value of [labeled](https://blueskyproject.io/bluesky/magics.html?highlight=label) items: [`%wa`]()

In [3]:
%wa

area_detector
  Local variable name                    Ophyd name (to be recorded as metadata)
  adsimdet                               adsimdet                              

motor
  Positioner                     Value       Low Limit   High Limit  Offset     
  m1                             0.0         -32000.0    32000.0     0.0        
  m10                            0.0         -32000.0    32000.0     0.0        
  m11                            0.0         -32000.0    32000.0     0.0        
  m12                            0.0         -32000.0    32000.0     0.0        
  m13                            0.0         -32000.0    32000.0     0.0        
  m14                            0.0         -32000.0    32000.0     0.0        
  m15                            0.0         -32000.0    32000.0     0.0        
  m16                            0.0         -32000.0    32000.0     0.0        
  m2                             0.0         -32000.0    32000.0     0.0        
  m3    

## **Who** is everybody?

Might be a good idea to know now what ophyd symbols are available.  The [apstools](https://apstools.readthedocs.io/en/latest/) package provides a [`listobjects()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listobjects) command that prints a table of all the known objects (in the global namespace of the session).  The columns provide the ophyd name (the name you use to call this in Python), the name of the ophyd structure, the EPICS PV (if relevant), and any labels (as used in `%wa` above).

Notably, the table includes:

* a simulated EPICS area detector (`adsimdet`), look for the `area_detector` label
* a simulated 16-channel EPICS scaler (`scaler1`), look for the `scalers` label
* some of the scaler channels are named (`I0`, `diode`, ...), look for the `counter` label
* a _noisy_ detector for use with motor `m1` to simulate a diffraction peak
* 16 simulated EPICS motors (`m1` .. `m16`), look for the `motor` label
* some calculation support (`calcs` & `calcouts`)
* a simulated shutter (`shutter`)
* a simulated temperature controller (`temperature`)
* details about the general purpose IOC (`gp_stats`)

In [4]:
listobjects()

=========== ================================ ============= =================
name        ophyd structure                  EPICS PV      label(s)         
=========== ================================ ============= =================
I0          EpicsSignalRO                    gp:scaler1.S2 channel counter  
adsimdet    MySimDetector                    ad:           area_detector    
calcouts    UserCalcoutDevice                gp:                            
calcs       UserCalcsDevice                  gp:                            
diode       EpicsSignalRO                    gp:scaler1.S4 channel counter  
gp_stats    IocInfoDevice                    gp:                            
m1          EpicsMotor                       gp:m1         motor            
m10         EpicsMotor                       gp:m10        motor            
m11         EpicsMotor                       gp:m11        motor            
m12         EpicsMotor                       gp:m12        motor            

## Let's work with the scaler

A scaler is a device that counts digital pulses from a pulse detector such
as a scintillation counter or from photodiodes or ionization chamber with
pulse chain electronics.  Scalers have
many channels, some of which might have no associated detector.  Our scaler
(`scaler1`) is a simulated device that records a random number of pulses in 
each channel.  We are only interested in the channels that have names provided
by users in the GUI screens.  In this screen for our scaler, only a few of the
channels are named:

![`scaler` GUI](resources/scaler.png "`scaler1` GUI")

Let's configure `scaler1` to report only the `diode` and `I0` channels (plus the count time channel which will *always* be included).  Keep in mind that the argument to this function is a Python list, so the channel names must be enclosed with `[]`.  The function does not return a result.  If something *is* printed, there is an error to be fixed.

NOTE:  To report _all_ named channels again, call the same function with argument in the parentheses.

In [16]:
scaler1.select_channels(["diode", "I0"])

The _easiest_ way to count the `scaler` object is to use the [`%ct`](https://blueskyproject.io/bluesky/magics.html?highlight=label#taking-a-reading-using-ct-post-v1-3-0) bluesky magic command, which counts all objects with the `detectors` label.

Note that the various magic commands are only available from the command line, not for use in a bluesky plan function.

In [13]:
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          5951.597596451602
I0                             3.0
diode                          4.0
scaler1_time                   1.1


Compare with the reading when _all_ channels are selected:

In [15]:
scaler1.select_channels()
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          5951.597596451602
timebase                       11000000.0
I0                             3.0
scint                          6.0
diode                          5.0
scaler1_time                   1.1


Now, select just the two channels again before continuing:

In [17]:
scaler1.select_channels(["diode", "I0"])

As noted before, the `%ct` command is only available from the command shell.

### use ophyd to count the scaler

We should learn how to use the underlying Python code to do the same steps.

The first step is to use pure ophyd methods to count and report, then use a bluesky plan to do the same thing.  The ophyd methods are `trigger`, `wait`, and `read`.  The `trigger` and `wait` methods can be chained together:

In [23]:
scaler1.trigger().wait()

Technically, we should `stage` and `unstage` the object.  **We'll use staging to control the count time of the scaler.**

The ophyd [`.stage()` method](https://nsls-ii.github.io/ophyd/generated/ophyd.device.BlueskyInterface.stage.html?highlight=stage#ophyd.device.BlueskyInterface.stage) prepares the object for its `.trigger()` method, while the `.unstage()` method returns the object's settings to the previous state before the `.stage()` method was called.

In [29]:
scaler1.stage()
scaler1.trigger().wait()
scaler1.unstage()

[Channels(prefix='gp:scaler1', name='scaler1_channels', parent='scaler1', read_attrs=['chan02', 'chan02.s', 'chan04', 'chan04.s'], configuration_attrs=['chan02', 'chan02.chname', 'chan02.preset', 'chan02.gate', 'chan04', 'chan04.chname', 'chan04.preset', 'chan04.gate']),
 ScalerCH(prefix='gp:scaler1', name='scaler1', read_attrs=['channels', 'channels.chan02', 'channels.chan02.s', 'channels.chan04', 'channels.chan04.s', 'time'], configuration_attrs=['channels', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'count_mode', 'delay', 'auto_count_delay', 'freq', 'preset_time', 'auto_count_time', 'egu'])]

Let's find out what happens when `scaler1` is staged.  That's controlled by the contents of a Python dictionary `.stage_sigs`:

In [24]:
scaler1.stage_sigs

OrderedDict()

It's empty, so nothing has been preconfigured for us.  Let's make sure that we get to pick the *counting time* (the time to accumulate pulses in the various channels), say 2.0 seconds, when we count here.

In [25]:
scaler1.stage_sigs["preset_time"] = 2
scaler1.stage_sigs

OrderedDict([('preset_time', 2)])

Show the counting time *before* we count, then `stage`, `trigger`, `wait`, `read`, `unstage`, then finally show the counting time  *after* we count:

In [32]:
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")
scaler1.stage()
scaler1.trigger().wait()
print(scaler1.read())
scaler1.unstage()
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")

Scaler configured to count for 1.0s
OrderedDict([('I0', {'value': 9.0, 'timestamp': 1614113769.207337}), ('diode', {'value': 8.0, 'timestamp': 1614113769.207337}), ('scaler1_time', {'value': 2.1, 'timestamp': 1614113748.906527})])
Scaler configured to count for 1.0s


The report from `.read()` includes both values and timestamps (in seconds since the Python [time](https://docs.python.org/3/library/time.html) epoch, UTC).  The structure is a Python dictionary.  This is the low-level method used to collect readings from any ophyd device.  We had to `print()` this since the return result from a command within a sequence is not returned at the end of the sequence, just the return result of the *final* command in the sequence.

See that the scaler counted for 2.1 seconds (a small bug in the scaler simulator it seems, always adds .1 to the count time!).  But before staging, the scaler was configured for 1.0 seconds, and after unstaging, the scaler returned to that value.

**That's how to control the counting time *for a scaler*.**  (Area detectors use different terms.  More on that later.)

<details>
    <summary>about <tt>scaler1_time</tt></summary>

     Of note is the key `scaler1_time` which is the name of the ophyd symbol `scaler1.time` as returned by `scaler1.time.name`:

        In [21]: scaler1.time.name
        Out [21]: 'scaler1_time'

</details>

### use bluesky (the package) to count the scaler

Now, use the bluesky [RunEngine](https://blueskyproject.io/bluesky/generated/bluesky.run_engine.RunEngine.html?highlight=runengine#bluesky.run_engine.RunEngine) (`RE`) to count `scaler1`.  We'll use the bluesky plan ([`bp`](https://blueskyproject.io/bluesky/plans.html?highlight=count#pre-assembled-plans)) called [`count()`](https://blueskyproject.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  To be consistent with the result returned from `%ct`, we'll include the `noisy` detector.

In [33]:
RE(bp.count([scaler1,noisy]))



Transient Scan ID: 138     Time: 2021-02-23 15:03:56
Persistent Unique Scan ID: '05806f7b-8d97-41f4-b1a3-f898ee92aee9'
New stream: 'baseline'
scaler1 [In progress. No progress bar available.]                              
scaler1 [In progress. No progress bar available.]                              
                                                                               
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |      noisy |         I0 |      diode |
+-----------+------------+------------+------------+------------+
|         1 | 15:03:59.6 | 5951.59760 |         10 |         10 |
+-----------+------------+------------+------------+------------+
generator count ['05806f7b'] (scan num: 138)


('05806f7b-8d97-41f4-b1a3-f898ee92aee9',)

There are many ways to view data from bluesky runs.  We'll pick one simple way, as a [dask](https://dask.org/) table since it is easy to display such structured content in a Jupyter notebook.

In [44]:
db.v2[-1].primary.to_dask()

<xarray.Dataset>
Dimensions:                                 (time: 1)
Coordinates:
  * time                                    (time) float64 1.614e+09
Data variables:
    noisy                                   (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    I0                                      (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    diode                                   (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    scaler1_time                            (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    noisy:noisy                             (time) float64 5.952e+03
    scaler1:scaler1_channels_chan02_chname  (time) <U2 'I0'
    scaler1:scaler1_channels_chan02_preset  (time) float64 0.0
    scaler1:scaler1_channels_chan02_gate    (time) <U1 'N'
    scaler1:scaler1_channels_chan04_chname  (time) <U5 'diode'
    scaler1:scaler1_channels_chan04_preset  (time) float64 0.0
    scaler1:scaler1_channels_chan04_gate    (time) <U1 'N'
    scaler1:scaler1_count_mode              (time) <U7 'OneShot'
    scaler1:scaler1_delay                   (time) float64 0.0
    scaler1:scaler1_auto_count_delay        (time) float64 0.0
    scaler1:scaler1_freq                    (time) float64 1e+07
    scaler1:scaler1_preset_time             (time) float64 2.0
    scaler1:scaler1_auto_count_time         (time) float64 1.0
    scaler1:scaler1_egu                     (time) <U1 ''
    seq_num                                 (time) int64 1
    uid                                     (time) <U36 'acfce394-7d32-48c1-9...

As a last action in this section, use the [`listruns()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listruns) command from *apstools* to show the (default: 20) most recent runs in the database.  The table shows a short version of the run's unique identifier (`short_uid`), and other more obvious columns of information, truncated to avoid lengthy output.  The name of the [databroker *catalog*](https://blueskyproject.io/databroker/index.html) (`class_2021_03`) is shown before the table.

In [45]:
listruns()

catalog name: class_2021_03
========= ========================== ======= ======= ========================================
short_uid date/time                  exit    scan_id command                                 
========= ========================== ======= ======= ========================================
05806f7   2021-02-23 15:03:56.999090 success 138     count(detectors=['scaler1', 'noisy'] ...
f1319ee   2021-02-23 12:17:44.589302 success 137     scan(detectors=['noisy'], num=11, ar ...
c03544f   2021-02-23 12:17:06.922094 success 136     count(detectors=['temperature'], num=5) 
00ff220   2021-02-23 12:16:58.645932 success 135     count(detectors=['temperature'], num=5) 
bdc1b90   2021-02-23 12:16:26.066357 success 134     count(detectors=['scaler1', 'noisy', ...
7e0862f   2021-02-23 12:14:59.922437 success 133     count(detectors=['scaler1', 'noisy', ...
fc5dcc3   2021-02-23 12:13:23.975526 success 132     count(detectors=['scaler1', 'noisy'] ...
========= ======================

## temperature _v_ time

In [7]:
# temperature.position
# temperature.get()
# temperature.readback.get()
# temperature.read()
# RE(bp.count([scaler1,noisy,temperature]))
# RE(bp.count([scaler1,noisy,temperature], num=5))
# RE(bp.count([temperature], num=5))
# RE(bp.count([temperature], num=5, delay=2))
# RE(bp.scan([noisy], temperature, 25, 35, 11))
# %mov temperature 25
# temperature.position
# %movr temperature 1
# RE(bps.mv(temperature, 25))
# RE(bps.mvr(temperature, 1))
# listdevice(temperature)

## Find a peak: detector _v_ motor

## Record an Image

----

---

[`%mov`](https://blueskyproject.io/bluesky/magics.html?highlight=label#moving-a-motor) (Magic command) : move a positioner